# 2. Optimisation demostration 

This notebook demostrates optimisation functionality. It covers:

2.1 Optimization Problem Setup
- Understand objective functions and constraints
- Configure optimization parameters

2.2 Single Optimization Run
- Configure PSO algorithm
- Run optimization with monitoring
- Analyze results and constraint violations

2.3 Multi-Run Analysis
- Statistical robustness testing
- Algorithm parameter sensitivity
- Convergence analysis

2.4 Advanced Scenarios
- Multiple constraint types
- Different objective functions
- Parameter tuning strategies



In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from typing import Dict, Any
import logging

# Add src to path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

print("=== TRANSIT OPTIMIZATION SETUP ===")
print("🚀 Starting self-contained optimization workflow")
print("📊 This notebook will prepare data from scratch and run PSO optimization")


"""
## 2.1 Data Preparation and Problem Setup

### GTFS Data Loading

First, we'll load the GTFS feed and transform it into optimization data structure.
This replicates the key steps from notebook 1 but focuses only on what's needed for optimization.


In [ ]:
from transit_opt.preprocessing.prepare_gtfs import GTFSDataPreparator

print("=== LOADING GTFS DATA ===")

# Create GTFS preparator with same settings as notebook 1
preparator = GTFSDataPreparator(
    gtfs_path='../data/external/study_area_gtfs_bus.zip',
    interval_hours=3,  # 8 periods per day
    date=None,  # Use full GTFS feed
    turnaround_buffer=1.15,  # 15% buffer
    max_round_trip_minutes=240.0,  # Maximum round-trip time
    no_service_threshold_minutes=480.0,  # Threshold for no-service mapping
    log_level="INFO"  # Less verbose than notebook 1
)

# Define allowed headways for optimization
allowed_headways = [5, 10, 15, 30, 60, 90, 120]

print(f"📋 Allowed headways: {allowed_headways} minutes")
print("🔄 Extracting optimization data...")

# Extract optimization data structure
opt_data = preparator.extract_optimization_data(allowed_headways)

print(f"\n✅ GTFS DATA PROCESSED:")
print(f"   📊 Routes: {opt_data['n_routes']}")
print(f"   ⏰ Time intervals: {opt_data['n_intervals']} (3h each)")
print(f"   🎯 Decision variables: {opt_data['decision_matrix_shape'][0]} × {opt_data['decision_matrix_shape'][1]} = {np.prod(opt_data['decision_matrix_shape'])}")
print(f"   🚗 Current peak fleet: {opt_data['constraints']['fleet_analysis']['total_current_fleet_peak']} vehicles")
print(f"   🔢 Headway choices: {opt_data['n_choices']} (including no-service)")



### Spatial Boundary Setup

Load the study area boundary for spatial filtering and analysis.
This ensures optimization focuses on the relevant geographic area.


In [ ]:
from transit_opt.optimisation.spatial.boundaries import StudyAreaBoundary

print("\n=== SPATIAL BOUNDARY SETUP ===")

# Load boundary geometry
boundary_gdf = gpd.read_file("../data/external/boundaries/study_area_boundary.geojson")
print(f"📍 Loaded boundary with {len(boundary_gdf)} feature(s)")

# Create study area boundary with buffer
study_boundary = StudyAreaBoundary(
    boundary_gdf=boundary_gdf,
    crs="EPSG:3857",  # Web Mercator for spatial analysis
    buffer_km=2.0     # 2km buffer around boundary
)

print(f"✅ Study area boundary created:")
print(f"   📐 CRS: {study_boundary.target_crs}")
print(f"   📏 Buffer: 2km")

### Optimization Problem Structure

Before diving into objectives and constraints, let's understand the mathematical structure
of our transit optimization problem.

In [ ]:
print("\n=== OPTIMIZATION PROBLEM STRUCTURE ===")
print("🔢 DECISION VARIABLES:")
print(f"   • Each route-interval combination = 1 optimization variable")
print(f"   • Matrix structure: {opt_data['decision_matrix_shape']} (routes × intervals)")
print(f"   • Total variables: {opt_data['decision_matrix_shape'][0] * opt_data['decision_matrix_shape'][1]}")
print(f"   • Each variable chooses from {opt_data['n_choices']} discrete headway options")

print(f"\n🎯 DISCRETE CHOICES:")
print(f"   • Continuous headways (e.g., 17.3 minutes) → Discrete choices (e.g., 15 minutes)")
print(f"   • Algorithms choose indices (0-{opt_data['n_choices']-1}) representing allowed headways")
print(f"   • Variable bounds: {opt_data['variable_bounds']} (choice indices)")

print("\n🕐 ALLOWED HEADWAYS:")
for i, headway in enumerate(opt_data['allowed_headways']):
    if headway >= 9000:
        print(f"   Index {i}: No Service ({headway})")
    else:
        print(f"   Index {i}: {headway:.0f} minutes")

print("\n⏰ TIME STRUCTURE:")
for i, (label, hours) in enumerate(zip(opt_data['intervals']['labels'], opt_data['intervals']['hours'])):
    print(f"   Interval {i}: {label} ({hours[0]:02d}:00-{hours[1]:02d}:00)")


### Objective Functions

Transit optimization typically involves competing objectives:

We'll focus on **Service Coverage** using spatial analysis. TODO: More complex objectives will be added later.

In [ ]:
from transit_opt.optimisation.objectives import HexagonalCoverageObjective

print("\n=== OBJECTIVE FUNCTION: SERVICE COVERAGE ===")
print("Hexagonal Coverage Objective measures spatial equity by:")
print("• Dividing study area into hexagonal zones")
print("• Calculating vehicle service per zone based on headways")
print("• Minimizing variance in service distribution")
print("• Lower variance = more equitable service coverage")

# Create coverage objective with boundary filtering
coverage_objective = HexagonalCoverageObjective(
    optimization_data=opt_data,
    spatial_resolution_km=3.0,
    crs="EPSG:3857",
    boundary=study_boundary,  # This filters spatial analysis to study area
    spatial_lag=True,
    alpha=0.2
)

print(f"\n📍 Spatial System Created:")
print(f"   🔸 Hexagonal zones: {len(coverage_objective.spatial_system.hex_grid)}")
print(f"   🚏 Transit stops (filtered): {len(coverage_objective.spatial_system.stops_gdf)}")
print(f"   📐 Zone size: ~{3.0} km diameter hexagons")

# Evaluate current service coverage
current_objective_value = coverage_objective.evaluate(opt_data['initial_solution'])
print(f"\n🎯 CURRENT SERVICE COVERAGE:")
print(f"   Objective value (variance): {current_objective_value:.4f}")
print(f"   Lower values = more equitable coverage")

# Get detailed analysis of current coverage
current_analysis = coverage_objective.get_detailed_analysis(opt_data['initial_solution'])
print(f"   Zones with service: {current_analysis['zones_with_service_average']}")
print(f"   Mean vehicles per zone: {current_analysis['total_vehicles_average']:.1f}")
print(f"   Coefficient of variation: {current_analysis['coefficient_of_variation_average']:.3f}")


### Constraint Types

Optimization constraints ensure solutions are operationally feasible:

1. **Fleet Total Constraint**: Limits peak vehicle requirements
2. **Fleet Per-Interval**: Limits vehicles needed in each time period  
3. **Minimum Service**: Ensures minimum service levels are maintained

All constraints work with headway decisions to calculate vehicle requirements.

In [ ]:
from transit_opt.optimisation.problems.base import (
    FleetTotalConstraintHandler,
    FleetPerIntervalConstraintHandler, 
    MinimumFleetConstraintHandler
)

print("\n=== CONSTRAINT SYSTEM OVERVIEW ===")
print("Constraints ensure optimization produces deployable solutions:\n")

# Show current fleet analysis
current_fleet = opt_data['constraints']['fleet_analysis']
print("📊 CURRENT FLEET ANALYSIS:")
print(f"   Peak vehicles needed: {current_fleet['total_current_fleet_peak']}")
print(f"   Fleet by interval: {current_fleet['current_fleet_by_interval'].tolist()}")
print(f"   Peak interval: {current_fleet['fleet_stats']['peak_interval']} ({opt_data['intervals']['labels'][current_fleet['fleet_stats']['peak_interval']]})")

print(f"\n🔒 CONSTRAINT EXAMPLES:")

# Fleet Total Constraint
print("1. Fleet Total Constraint:")
print("   • Limits peak vehicles across all time periods")
print("   • Example: ≤ 120% of current peak fleet")
print(f"   • Current peak: {current_fleet['total_current_fleet_peak']} vehicles")
print(f"   • 120% limit: {int(current_fleet['total_current_fleet_peak'] * 1.2)} vehicles")

print("\n2. Fleet Per-Interval Constraint:")
print("   • Limits vehicles needed in each 3-hour period")  
print("   • Prevents unrealistic concentration in one time period")
print("   • Example: ≤ 150% of current interval fleet")

print("\n3. Minimum Fleet Constraint:")
print("   • Ensures minimum service levels are maintained")
print("   • Prevents optimization from eliminating essential routes")
print("   • Example: ≥ 80% of current system-wide service")

# Show service coverage by time interval
print(f"\n📊 CURRENT SERVICE ACTIVITY:")
for i, label in enumerate(opt_data['intervals']['labels']):
    active_routes = np.sum(~np.isnan(opt_data['routes']['current_headways'][:, i]))
    coverage_pct = 100 * active_routes / opt_data['n_routes']
    fleet_needed = current_fleet['current_fleet_by_interval'][i]
    print(f"   {label}: {active_routes}/{opt_data['n_routes']} routes ({coverage_pct:.0f}%), {fleet_needed:.0f} vehicles")



### Problem Configuration

We'll use the configuration manager system to set up PSO optimization with multiple constraints.

In [1]:
# Update the configuration cell (around line 294) with penalty method options:

print("=== OPTIMIZATION CONFIGURATION ===")
print("🎯 PENALTY METHOD vs HARD CONSTRAINTS")
print("Choose between two constraint handling approaches:")
print("• Hard Constraints: Reject infeasible solutions completely")
print("• Penalty Method: Add constraint violations to objective (explore infeasible regions)")

# Configuration with PENALTY METHOD (recommended for exploration)
config_penalty = {
    'problem': {
        'objective': {
            'type': 'HexagonalCoverageObjective',
            'spatial_resolution_km': 3.0,
            'crs': 'EPSG:3857',
            'boundary_file': '../data/external/boundaries/study_area_boundary.geojson',
            'boundary_buffer_km': 2.0
        },
        'constraints': [
            {
                'type': 'FleetTotalConstraintHandler',
                'baseline': 'current_peak',
                'tolerance': 0.2,  # 20% increase allowed
                'measure': 'peak'
            },
            {
                'type': 'FleetPerIntervalConstraintHandler',
                'baseline': 'current_by_interval',
                'tolerance': 0.3,  # 30% increase per interval
                'allow_borrowing': False
            },
            {
                'type': 'MinimumFleetConstraintHandler',
                'min_fleet_fraction': 0.3,  # Maintain 30% of current service
                'level': 'system',
                'measure': 'peak', 
                'baseline': 'current_peak'
            }
        ],
        # 🔧 NEW: Constraint-specific penalty weights
        'penalty_weights': {
            'fleet_total': 2000.0,        # High penalty for budget violations
            'fleet_per_interval': 1000.0, # Medium penalty for operational violations
            'minimum_fleet': 5000.0       # Very high penalty for service cuts
        }
    },
    'optimization': {
        'algorithm': {
            'type': 'PSO',
            'pop_size': 50,
            'inertia_weight': 0.9,
            'inertia_weight_final': 0.4,
            'cognitive_coeff': 2.0,
            'social_coeff': 2.0,
            # 🔧 NEW: Penalty method configuration
            'use_penalty_method': True,      # Enable penalty method
            'penalty_weight': 1500.0,        # Default penalty for unspecified constraints
            'adaptive_penalty': True,        # Increase penalties over generations
            'penalty_increase_rate': 1.3     # 30% penalty increase per generation
        },
        'termination': {
            'max_generations': 100  # More generations for penalty method convergence
        },
        'monitoring': {
            'progress_frequency': 10,
            'save_history': True,
            'detailed_logging': True
        }
    }
}

# Alternative: Hard constraints configuration (traditional approach)
config_hard = {
    'problem': {
        'objective': {
            'type': 'HexagonalCoverageObjective',
            'spatial_resolution_km': 3.0,
            'crs': 'EPSG:3857',
            'boundary_file': '../data/external/boundaries/study_area_boundary.geojson',
            'boundary_buffer_km': 2.0
        },
        'constraints': [
            {
                'type': 'FleetTotalConstraintHandler',
                'baseline': 'current_peak',
                'tolerance': 0.25,  # More lenient for hard constraints
                'measure': 'peak'
            },
            {
                'type': 'MinimumFleetConstraintHandler',
                'min_fleet_fraction': 0.25,  # More lenient for hard constraints
                'level': 'system',
                'measure': 'peak',
                'baseline': 'current_peak'
            }
        ]
        # No penalty_weights section - not used for hard constraints
    },
    'optimization': {
        'algorithm': {
            'type': 'PSO',
            'pop_size': 50,
            'inertia_weight': 0.9,
            'inertia_weight_final': 0.4,
            'cognitive_coeff': 2.0,
            'social_coeff': 2.0,
            # Hard constraints mode (default)
            'use_penalty_method': False      # Traditional constraint handling
        },
        'termination': {
            'max_generations': 50   # Fewer generations often sufficient for hard constraints
        },
        'monitoring': {
            'progress_frequency': 10,
            'save_history': True,
            'detailed_logging': True
        }
    }
}

# 🔧 CHOOSE CONFIGURATION MODE
USE_PENALTY_METHOD = True  # Set to False to try hard constraints

config = config_penalty if USE_PENALTY_METHOD else config_hard
method_name = "Penalty Method" if USE_PENALTY_METHOD else "Hard Constraints"

print(f"\n🤖 SELECTED: {method_name}")
print(f"   Algorithm: Particle Swarm Optimization (PSO)")
print(f"   Population size: {config['optimization']['algorithm']['pop_size']} particles")
print(f"   Max generations: {config['optimization']['termination']['max_generations']}")

if USE_PENALTY_METHOD:
    print(f"   Penalty weights: Budget={config['problem']['penalty_weights']['fleet_total']}, ")
    print(f"                   Operational={config['problem']['penalty_weights']['fleet_per_interval']}, ")
    print(f"                   Service={config['problem']['penalty_weights']['minimum_fleet']}")
    print(f"   Adaptive penalties: {config['optimization']['algorithm']['adaptive_penalty']}")
    if config['optimization']['algorithm']['adaptive_penalty']:
        rate = config['optimization']['algorithm']['penalty_increase_rate']
        print(f"   Penalty increase: {rate}x per generation (~{rate**50:.0f}x after 50 gens)")

print(f"\n🎯 Objective: Service Coverage (Spatial Equity)")
print(f"   Minimize variance in vehicle distribution across hexagonal zones")

print(f"\n🔒 Constraints: {len(config['problem']['constraints'])} active")
for i, constraint in enumerate(config['problem']['constraints'], 1):
    constraint_type = constraint['type'].replace('ConstraintHandler', '')
    tolerance = constraint.get('tolerance', constraint.get('min_fleet_fraction', 'N/A'))
    print(f"   {i}. {constraint_type}: {tolerance}")

print(f"\n📊 Constraint Handling Method:")
if USE_PENALTY_METHOD:
    print("   🎯 PENALTY METHOD:")
    print("   • Violations added as penalties to objective function")
    print("   • Allows temporary exploration of infeasible regions")
    print("   • Penalties increase over generations → convergence to feasibility")
    print("   • Better exploration, potentially better final solutions")
else:
    print("   🚦 HARD CONSTRAINTS:")
    print("   • Infeasible solutions completely rejected")
    print("   • Search restricted to feasible region only")
    print("   • Direct constraint enforcement")
    print("   • More predictable but potentially limited exploration")

=== OPTIMIZATION CONFIGURATION ===
🎯 PENALTY METHOD vs HARD CONSTRAINTS
Choose between two constraint handling approaches:
• Hard Constraints: Reject infeasible solutions completely
• Penalty Method: Add constraint violations to objective (explore infeasible regions)

🤖 SELECTED: Penalty Method
   Algorithm: Particle Swarm Optimization (PSO)
   Population size: 50 particles
   Max generations: 100
   Penalty weights: Budget=2000.0, 
                   Operational=1000.0, 
                   Service=5000.0
   Adaptive penalties: True
   Penalty increase: 1.3x per generation (~497929x after 50 gens)

🎯 Objective: Service Coverage (Spatial Equity)
   Minimize variance in vehicle distribution across hexagonal zones

🔒 Constraints: 3 active
   1. FleetTotal: 0.2
   2. FleetPerInterval: 0.3
   3. MinimumFleet: 0.3

📊 Constraint Handling Method:
   🎯 PENALTY METHOD:
   • Violations added as penalties to objective function
   • Allows temporary exploration of infeasible regions
   • Penalties 

In [ ]:
from transit_opt.optimisation.config.config_manager import OptimizationConfigManager
from transit_opt.optimisation.runners.pso_runner import PSORunner

# Create optimization configuration
config = {
    'problem': {
        'objective': {
            'type': 'HexagonalCoverageObjective',
            'spatial_resolution_km': 3.0,
            'crs': 'EPSG:3857',
            'boundary_file': '../data/external/boundaries/study_area_boundary.geojson',
            'boundary_buffer_km': 2.0
        },
        'constraints': [
            # Fleet total constraint: Allow 20% increase
            {
                'type': 'FleetTotalConstraintHandler',
                'baseline': 'current_peak',
                'tolerance': 0.20,  # 20% increase allowed
                'measure': 'peak'
            },
            # Per-interval constraint: More lenient
            # {
            #     'type': 'FleetPerIntervalConstraintHandler', 
            #     'baseline': 'current_by_interval',
            #     'tolerance': 0.5,  # allowed % increase per interval 
            #     'allow_borrowing': False  # Explicit setting

            # },
            # Minimum service: Maintain 70% of current
            {
                'type': 'MinimumFleetConstraintHandler',
                'min_fleet_fraction': 0.30,
                'level': 'system', 
                'measure': 'peak',
                'baseline': 'current_peak'
            }
        ]
    },
    'optimization': {
        'algorithm': {
            'type': 'PSO',
            'pop_size': 25,  # Population size
            'inertia_weight': 0.9,  # Initial inertia
            #'inertia_weight_final': 0.4,  # Final inertia (adaptive)
            'cognitive_coeff': 2.0,  # Cognitive coefficient
            'social_coeff': 2.0  # Social coefficient
        },
        'termination': {
            'max_generations':10 # Reduced for quicker testing 
        },
        'monitoring': {
            'progress_frequency': 10,
            'save_history': True,
            'detailed_logging': True
        }
    }
}

print("=== OPTIMIZATION CONFIGURATION ===")
print("🤖 Algorithm: Particle Swarm Optimization (PSO)")
print(f"   Population size: {config['optimization']['algorithm']['pop_size']} particles")
print(f"   Max generations: {config['optimization']['termination']['max_generations']}")
#print(f"   Adaptive inertia: {config['optimization']['algorithm']['inertia_weight']} → {config['optimization']['algorithm']['inertia_weight_final']}")

print(f"\n🎯 Objective: Service Coverage (Spatial Equity)")
print(f"   Minimize variance in vehicle distribution")
print(f"   Spatial resolution: {config['problem']['objective']['spatial_resolution_km']} km hexagons")

print(f"\n🔒 Constraints: {len(config['problem']['constraints'])} active")
for i, constraint in enumerate(config['problem']['constraints'], 1):
    constraint_type = constraint['type'].replace('Handler', '').replace('Constraint', '')
    print(f"   {i}. {constraint_type}: {constraint.get('tolerance', constraint.get('min_fleet_fraction', 'N/A'))}")


## 2.2 Single Optimization Run

Let's run a single PSO optimization to see how the algorithm improves service coverage.

In [ ]:
# Create and run PSO
print("🚀 STARTING PSO OPTIMIZATION")
print("This may take 2-5 minutes depending on problem size...\n")

config_manager = OptimizationConfigManager(config_dict=config)
pso_runner = PSORunner(config_manager)

# Run optimization
result = pso_runner.optimize(opt_data)

print(f"\n✅ OPTIMIZATION COMPLETED")
print(f"⏱️  Total time: {result.optimization_time:.1f} seconds")
print(f"📊 Generations: {result.generations_completed}")
print(f"🎯 Best objective: {result.best_objective:.6f}")
print(f"📈 Improvement: {((current_objective_value - result.best_objective) / current_objective_value * 100):+.1f}%")

# Check constraint feasibility  
violations = result.constraint_violations
if violations['feasible']:
    print("✅ Solution is feasible (satisfies all constraints)")
else:
    print(f"❌ Solution violates {violations['total_violations']} constraints")
    print("   Constraint violation details:")
    for detail in violations['violation_details']:
        if detail['violation'] < -0.001:  # Threshold for numerical precision
            print(f"   • Constraint {detail['constraint_idx']}: {detail['violation']:.3f}")


# Debugging start

In [ ]:
# Add this to test the inertia weight schedule
from transit_opt.optimisation.runners.pso_runner import AdaptivePSO

print("🔍 TESTING ADAPTIVE INERTIA WEIGHT:")
pso = AdaptivePSO(inertia_weight=0.9, inertia_weight_final=0.4)

max_gen = 10
for gen in [0, 1, 2, 5, 8, 9]:
    weight = pso._calculate_adaptive_weight(gen, max_gen)
    print(f"   Generation {gen}: {weight:.3f}")

print("\n   Expected: 0.9 → 0.4 (decreasing)")

In [ ]:
print("🔍 DETAILED CONSTRAINT DIAGNOSIS:")

# Test different tolerances with current solution (should always pass)
test_tolerances = [0.2, 1.0, 5.0, 10.0, 100.0]

for tol in test_tolerances:
    print(f"\n📊 Testing tolerance = {tol} ({tol*100:.0f}% increase):")
    
    config_test = {
        'baseline': 'current_by_interval',
        'tolerance': tol
    }
    
    handler_test = FleetPerIntervalConstraintHandler(config_test, opt_data)
    
    # Get limits
    baseline_vals = np.array(opt_data['constraints']['fleet_analysis']['current_fleet_by_interval'])
    limits = handler_test._get_interval_limits()
    
    print(f"   Baseline: {baseline_vals[:3]}... (first 3 intervals)")
    print(f"   Limits: {limits[:3]}... (first 3 intervals)")
    print(f"   Multiplier: {(limits[0]/baseline_vals[0]):.2f}x")
    
    # Test with CURRENT solution (should always pass)
    violations = handler_test.evaluate(opt_data['initial_solution'])
    max_violation = np.max(violations)
    num_violated = np.sum(violations > 0)
    
    print(f"   Current solution max violation: {max_violation:.1f}")
    print(f"   Intervals violated: {num_violated}/{len(violations)}")
    
    if num_violated > 0:
        print("   ❌ PROBLEM: Current solution violates its own baseline!")
        violating_intervals = np.where(violations > 0)[0][:3]  # First 3
        for i in violating_intervals:
            print(f"      Interval {i}: needs {violations[i] + limits[i]:.1f}, limit {limits[i]:.1f}")
    else:
        print("   ✅ Current solution satisfies constraint")

In [ ]:
print("🔍 BASELINE CONSISTENCY CHECK:")

# Get baseline from data structure
baseline_from_data = np.array(opt_data['constraints']['fleet_analysis']['current_fleet_by_interval'])
print(f"Baseline from data: {baseline_from_data}")

# Calculate what current solution actually requires
from transit_opt.optimisation.utils.fleet_calculations import calculate_fleet_requirements

try:
    current_fleet_calc = calculate_fleet_requirements(
        headways_matrix=opt_data['initial_solution'],
        round_trip_times=opt_data['routes']['round_trip_times'],
        operational_buffer=1.15,
        no_service_threshold=480,
        allowed_headways=opt_data['allowed_headways'],
        no_service_index=opt_data.get('no_service_index')
    )
    
    actual_fleet_by_interval = current_fleet_calc['fleet_per_interval']
    print(f"Actual from calculation: {actual_fleet_by_interval}")
    
    # Compare
    difference = actual_fleet_by_interval - baseline_from_data
    print(f"Difference: {difference}")
    print(f"Max difference: {np.max(np.abs(difference)):.1f} vehicles")
    
    if np.max(np.abs(difference)) > 1.0:
        print("❌ MAJOR INCONSISTENCY: Baseline and actual fleet don't match!")
        print("This explains why low tolerances don't work.")
    else:
        print("✅ Baseline and actual fleet are consistent")
        
except Exception as e:
    print(f"❌ Fleet calculation failed: {e}")
    print("This confirms the division-by-zero bug is causing constraint issues")

In [ ]:
print("🔍 CHECKING FLEET CALCULATION RETURN STRUCTURE:")

try:
    result = calculate_fleet_requirements(
        headways_matrix=opt_data['initial_solution'],
        round_trip_times=opt_data['routes']['round_trip_times'],
        operational_buffer=1.15,
        no_service_threshold=480,
        allowed_headways=opt_data['allowed_headways'],
        no_service_index=opt_data.get('no_service_index')
    )
    
    print("✅ Fleet calculation succeeded!")
    print(f"   Return type: {type(result)}")
    print(f"   Available keys: {list(result.keys()) if isinstance(result, dict) else 'Not a dict'}")
    
    if isinstance(result, dict):
        for key, value in result.items():
            print(f"   {key}: {type(value)} - {np.array(value).shape if hasattr(value, 'shape') else 'scalar'}")
            
except Exception as e:
    print(f"❌ Fleet calculation failed: {e}")
    print(f"   Error type: {type(e).__name__}")
    import traceback
    traceback.print_exc()

In [ ]:
# Add this diagnostic to see what's happening
print("🔍 HEADWAY MAPPING ANALYSIS:")
print("Real GTFS headways vs. Discrete choices:")

for route_idx in range(min(7, opt_data['n_routes'])):  # First 5 routes
    gtfs_headways = opt_data['routes']['current_headways'][route_idx]
    initial_choices = opt_data['initial_solution'][route_idx]
    
    print(f"\nRoute {route_idx}:")
    for interval_idx in range(opt_data['n_intervals']):
        real_hw = gtfs_headways[interval_idx] 
        choice_idx = initial_choices[interval_idx]
        discrete_hw = opt_data['allowed_headways'][choice_idx] if choice_idx < len(opt_data['allowed_headways']) else 'No Service'
        
        print(f"  Interval {interval_idx}: {real_hw:.1f}min → {discrete_hw}")

# Debugging end

### Solution Analysis

Let's analyze how the optimization changed the transit service patterns.

In [ ]:
print("=== SOLUTION ANALYSIS ===")

# Get detailed analysis from objective function
solution_analysis = coverage_objective.get_detailed_analysis(result.best_solution)
current_analysis = coverage_objective.get_detailed_analysis(opt_data['initial_solution'])

print("📊 SERVICE COVERAGE COMPARISON:")
print(f"                     Before      After       Change")
print(f"   Variance:         {current_analysis['variance_average']:.4f}    {solution_analysis['variance_average']:.4f}    {((solution_analysis['variance_average'] - current_analysis['variance_average']) / current_analysis['variance_average'] * 100):+.1f}%")
print(f"   Mean vehicles:    {current_analysis['total_vehicles_average']:.1f}      {solution_analysis['total_vehicles_average']:.1f}       {((solution_analysis['total_vehicles_average'] - current_analysis['total_vehicles_average']) / current_analysis['total_vehicles_average'] * 100):+.1f}%")
print(f"   Zones with service: {current_analysis['zones_with_service_average']}        {solution_analysis['zones_with_service_average']}         {solution_analysis['zones_with_service_average'] - current_analysis['zones_with_service_average']:+.0f}")
print(f"   Coeff of variation: {current_analysis['coefficient_of_variation_average']:.3f}     {solution_analysis['coefficient_of_variation_average']:.3f}     {((solution_analysis['coefficient_of_variation_average'] - current_analysis['coefficient_of_variation_average']) / current_analysis['coefficient_of_variation_average'] * 100):+.1f}%")

# Fleet requirement comparison
from transit_opt.optimisation.utils import calculate_fleet_requirements

current_fleet_reqs = calculate_fleet_requirements(
    opt_data['initial_solution'], 
    opt_data['routes']['round_trip_times'], 
    opt_data
)

optimized_fleet_reqs = calculate_fleet_requirements(
    result.best_solution,
    opt_data['routes']['round_trip_times'], 
    opt_data  
)

print(f"\n🚗 FLEET REQUIREMENTS COMPARISON:")
print(f"   Current peak fleet: {np.max(current_fleet_reqs['fleet_by_interval']):.1f} vehicles")
print(f"   Optimized peak fleet: {np.max(optimized_fleet_reqs['fleet_by_interval']):.1f} vehicles") 
print(f"   Fleet change: {((np.max(optimized_fleet_reqs['fleet_by_interval']) - np.max(current_fleet_reqs['fleet_by_interval'])) / np.max(current_fleet_reqs['fleet_by_interval']) * 100):+.1f}%")



### Convergence Analysis

Analyze how PSO converged to the optimal solution.

In [ ]:
# Plot convergence
history = result.optimization_history

if history:
    generations = [gen['generation'] for gen in history]
    best_objectives = [gen['best_objective'] for gen in history]
    mean_objectives = [gen['mean_objective'] for gen in history]
    
    plt.figure(figsize=(12, 6))
    
    # Plot convergence
    plt.subplot(1, 2, 1)
    plt.plot(generations, best_objectives, 'b-', label='Best', linewidth=2)
    plt.plot(generations, mean_objectives, 'r--', label='Population Mean', alpha=0.7)
    plt.xlabel('Generation')
    plt.ylabel('Objective Value (Variance)')
    plt.title('PSO Convergence')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot improvement rate
    plt.subplot(1, 2, 2)
    if len(best_objectives) > 1:
        improvements = [best_objectives[i] - best_objectives[i-1] for i in range(1, len(best_objectives))]
        plt.plot(generations[1:], improvements, 'g-', linewidth=2)
        plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    plt.xlabel('Generation')
    plt.ylabel('Objective Improvement')  
    plt.title('Generation-to-Generation Improvement')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Convergence statistics
    final_improvement = best_objectives[-1] - best_objectives[0]
    print(f"\n📈 CONVERGENCE STATISTICS:")
    print(f"   Initial objective: {best_objectives[0]:.6f}")
    print(f"   Final objective: {best_objectives[-1]:.6f}")
    print(f"   Total improvement: {final_improvement:.6f} ({(final_improvement/best_objectives[0]*100):+.2f}%)")
    
    # Find when major improvements stopped
    if len(improvements) > 10:
        recent_improvements = improvements[-10:]
        avg_recent_improvement = np.mean(recent_improvements)
        print(f"   Recent improvement rate: {avg_recent_improvement:.8f} per generation")
        if abs(avg_recent_improvement) < abs(final_improvement) * 0.01:
            print("   🏁 Algorithm appears to have converged (minimal recent progress)")
        else:
            print("   🔄 Algorithm still making progress when terminated")


### Solution Visualization  

Visualize the spatial changes in service coverage.

In [ ]:

print("=== SPATIAL COVERAGE VISUALIZATION ===")

# Visualize current vs optimized coverage
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Current solution
coverage_objective.spatial_system.visualize_spatial_coverage(
    solution_matrix=opt_data['initial_solution'],
    optimization_data=opt_data,
    figsize=None,  # Use subplot
    ax=axes[0],
    show_stops=True
)
axes[0].set_title(f'Current Service Coverage\nVariance: {current_analysis["variance_average"]:.4f}', fontsize=14)

# Optimized solution  
coverage_objective.spatial_system.visualize_spatial_coverage(
    solution_matrix=result.best_solution,
    optimization_data=opt_data,
    figsize=None,  # Use subplot
    ax=axes[1], 
    show_stops=True
)
axes[1].set_title(f'Optimized Service Coverage\nVariance: {solution_analysis["variance_average"]:.4f}', fontsize=14)

plt.tight_layout()
plt.show()

variance_improvement = ((current_analysis['variance_average'] - solution_analysis['variance_average']) / current_analysis['variance_average']) * 100
print(f"🎯 Spatial equity improvement: {variance_improvement:+.1f}% reduction in coverage variance")

print("\n✅ Single optimization analysis complete!")